<a href="https://colab.research.google.com/github/jesusvillota/CSS_DataScience_2025/blob/main/Session2/2_1_BIS_Scraper_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="max-width: 880px; margin: 20px auto 22px; padding: 0px; border-radius: 18px; border: 1px solid #e5e7eb; background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%); box-shadow: 0 8px 26px rgba(0,0,0,0.06); overflow: hidden;">

  <!-- Banner Header -->
  <div style="padding: 34px 32px 14px; text-align: center; line-height: 1.38;">
    <div style="font-size: 13px; letter-spacing: 0.14em; text-transform: uppercase; color: #6b7280; font-weight: bold; margin-bottom: 5px;">
      Session #2
    </div>
    <div style="font-size: 29px; font-weight: 800; color: #14276c; margin-bottom: 4px;">
      Scraping Central Bank Speeches from the BIS
    </div>
    <div style="font-size: 29px; font-weight: 800; color: #14276c; margin-bottom: 4px;">
      Part II
    </div>
    <div style="font-size: 16.5px; color: #374151; font-style: italic; margin-bottom: 0;">
      Using Textual Data in Empirical Monetary Economics
    </div>
  </div>

  <!-- Logo Section -->
  <div style="background: none; text-align: center; margin: 30px 0 10px;">
    <img src="https://www.cemfi.es/images/Logo-Azul.png" alt="CEMFI Logo" style="width: 158px; filter: drop-shadow(0 2px 12px rgba(56,84,156,0.05)); margin-bottom: 0;">
  </div>

  <!-- Name -->
  <div style="font-family: 'Times New Roman', Times, serif; color: #38549c; text-align: center; font-size: 1.22em; font-weight: bold; margin-bottom: 0px;">
    Jesus Villota Miranda © 2025
  </div>

  <!-- Contact info -->
  <div style="font-family: 'Times New Roman', Times, serif; color: #38549c; text-align: center; font-size: 1em; margin-top: 7px; margin-bottom: 20px;">
    <a href="mailto:jesus.villota@cemfi.edu.es" style="color: #38549c; text-decoration: none; margin-right:8px;" title="Email">
      <!-- Email logo -->
      <!-- <img src="https://cdn-icons-png.flaticon.com/512/11679/11679732.png" alt="Email" style="width:18px; vertical-align:middle; margin-right:5px;"> -->
      jesus.villota@cemfi.edu.es
    </a>
    <span style="color:#9fa7bd;">|</span>
    <a href="https://www.linkedin.com/in/jesusvillotamiranda/" target="_blank" style="color: #38549c; text-decoration: none; margin-left:7px;" title="LinkedIn">
      <!-- LinkedIn logo -->
      <!-- <img src="https://1.bp.blogspot.com/-onvhHUdW1Us/YI52e9j4eKI/AAAAAAAAE4c/6s9wzOpIDYcAo4YmTX1Qg51OlwMFmilFACLcBGAsYHQ/s1600/Logo%2BLinkedin.png" alt="LinkedIn" style="width:22px; vertical-align:middle; margin-right:5px;"> -->
      LinkedIn
    </a>
  </div>
</div>


**IMPORTANT**: **Are you running this notebook in Google Colab?**

- If so, please make sure that in the cell below `running_in_colab` is set to `True`

- And, of course,  make sure to **run the cell**!

In [6]:
# ARE YOU RUNNING THIS IN GOOGLE COLAB? If YES, type True below
running_in_colab = False

# **1. Initial Setup**

In this section, we set up the necessary parameters and environment for our web scraping project. We'll:

1. Define constants like base URL, download directory, and date ranges
2. Import required libraries (`requests`, `selenium`, etc.)
3. Create directories for storing downloaded PDFs and extracted text
4. Define a utility function to generate BIS (Bank for International Settlements) search URLs with pagination parameters

This setup is crucial as it establishes the foundation for our scraping process, ensuring we have all the necessary tools and configurations in place before proceeding with data collection.

In [7]:
# --- Conditional install ---
if running_in_colab:
    # Install selenium if running in Colab
    !pip install selenium PyPDF2

# --- Imports ---
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import PyPDF2
import os
from pathlib import Path

# --- BIS Link generator ---
def bis_link(initial_date, final_date, page, page_length): 
    index_url = (
        f"https://www.bis.org/cbspeeches"
        f"?fromDate={initial_date}"
        f"&tillDate={final_date}"
        f"&cbspeeches_page={page}"
        f"&cbspeeches_page_length={page_length}"
    )
    return index_url

In [8]:
# --- Setup params ---
BASE_URL = "https://www.bis.org"
OUTPUT_BASE = Path("output/2_2/")
DOWNLOAD_DIR = OUTPUT_BASE / "downloads"
TEXT_DIR = OUTPUT_BASE / "texts"
INITIAL_DATE = "01/01/2000"
FINAL_DATE = "11/08/2025"
PAGE_LENGTH = 10
MAX_PAGE = 5

if running_in_colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  TEXT_DIR = Path(f'/content/gdrive/My Drive/{TEXT_DIR}')
  DOWNLOAD_DIR = Path(f'/content/gdrive/My Drive/{DOWNLOAD_DIR}')
  print(DOWNLOAD_DIR)
else:
  DOWNLOAD_DIR = Path("../") / DOWNLOAD_DIR
  TEXT_DIR = Path("../") / TEXT_DIR

DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)
TEXT_DIR.mkdir(parents=True, exist_ok=True)

# **2. Scraping PDF files with Selenium**

This section demonstrates how to use `Selenium` WebDriver to automate the process of finding and downloading PDF files from the BIS website. The process involves:

1. Initializing a Chrome WebDriver to automate browser interactions
2. Navigating through multiple pages of search results using our previously defined pagination function
3. Finding and extracting links to speech review pages
4. Visiting each review page to locate the PDF download links
5. Downloading the PDFs and saving them to our local directory

`Selenium` is particularly useful here because it can handle JavaScript-rendered content that might not be accessible with simple HTTP requests. This allows us to interact with dynamic elements on the page and navigate through the site's structure programmatically.

<!-- > **IMPORTANT NOTE**: If we all run the cell below in class, we may overload the BIS server. Please, be mindful of this. Consider running this script at home instead of now. -->

In [9]:
if running_in_colab:
    from selenium.webdriver.chrome.options import Options
    chrome_options = Options()
    chrome_options.add_argument('--headless') # Run in headless mode
    chrome_options.add_argument('--no-sandbox') # Bypass OS security model
    chrome_options.add_argument('--disable-dev-shm-usage') # Overcome limited resource problems
    driver = webdriver.Chrome(options=chrome_options)
else: 
    driver = webdriver.Chrome()

# Helper to sanitize parts for filenames
import re
import unicodedata

def sanitize_part(s: str) -> str:
    s = (s or '').strip().replace(' ', '_')
    # Keep Unicode accents; remove unsafe characters
    s = unicodedata.normalize('NFC', s)
    s = re.sub(r"[^0-9A-Za-z_\-\u00C0-\u024F\u1E00-\u1EFF]", "_", s)
    s = re.sub(r"_+", "_", s)
    return s

for i in range(1, MAX_PAGE + 1):
    index_url = bis_link(INITIAL_DATE, FINAL_DATE, i, PAGE_LENGTH)
    print(f"\n==============[ Processing page {i} ]==============")
    driver.get(index_url)
    time.sleep(5)  # Wait for JS

    try:
        container = driver.find_element(By.ID, "cbspeeches_list")
        review_links = container.find_elements(By.CSS_SELECTOR, "a.dark[href^='/review/']")
        review_hrefs = [link.get_attribute("href") for link in review_links]
        print(f"---> 🔗 Found {len(review_hrefs)} review links on page {i}")
    except Exception as e:
        print(f"---> ❌ Could not find review links on page {i}: {e}")
        continue

    # --- Iterate over each review link ---
    for review_url in review_hrefs:
        print(f"\n🌐 Visiting: {review_url}")
        driver.get(review_url)
        time.sleep(2)  # Wait for detail page JS (adjust if necessary)

        # Extract date and author from the detail page
        date_text = None
        author_text = None
        try:
            date_text = driver.find_element(By.XPATH, '//*[@id="center"]/div[2]/div[2]/div[1]/div').text.strip()
        except Exception as e:
            print(f"⚠️ Could not extract date: {e}")
        try:
            author_text = driver.find_element(By.XPATH, '//*[@id="center"]/div[2]/div[2]/div[2]/div/a').text.strip()
        except Exception as e:
            print(f"⚠️ Could not extract author: {e}")

        filename_base = None
        if date_text and author_text:
            date_clean = sanitize_part(date_text)
            author_clean = sanitize_part(author_text)
            filename_base = f"{date_clean}-{author_clean}"
            print(f"🧾 Using filename base: {filename_base}")

        # Look for pdf link on the detail page
        try:
            pdf_link = driver.find_element(By.CSS_SELECTOR, "a.pdftitle_link[href$='.pdf']")
            pdf_href = pdf_link.get_attribute("href")
            if not pdf_href.startswith("http"):
                pdf_href = BASE_URL + pdf_href
            print("📄 PDF found:", pdf_href)

            # Download the PDF
            response = requests.get(pdf_href)
            if filename_base:
                filename = f"{filename_base}.pdf"
            else:
                filename = Path(pdf_href).name
            save_path = DOWNLOAD_DIR / filename

            # Avoid overwriting if file exists
            if save_path.exists():
                stem = save_path.stem
                suffix = save_path.suffix
                counter = 1
                while (DOWNLOAD_DIR / f"{stem}_{counter}{suffix}").exists():
                    counter += 1
                save_path = DOWNLOAD_DIR / f"{stem}_{counter}{suffix}"

            with open(save_path, "wb") as f:
                f.write(response.content)
            print(f"📩 Downloaded PDF to {save_path}")
        except Exception as e:
            print(f"❌ No PDF found or error: {e}")

driver.quit()


==============[ Processing page 1 ]==============
---> 🔗 Found 10 review links on page 1

🌐 Visiting: https://www.bis.org/review/r250728g.htm
🧾 Using filename base: 05_August_2025-Swaminathan_J
📄 PDF found: https://www.bis.org/review/r250728g.pdf
📩 Downloaded PDF to ../output/2_2/downloads/05_August_2025-Swaminathan_J.pdf

🌐 Visiting: https://www.bis.org/review/r250728f.htm
🧾 Using filename base: 05_August_2025-Michele_Bullock
📄 PDF found: https://www.bis.org/review/r250728f.pdf
📩 Downloaded PDF to ../output/2_2/downloads/05_August_2025-Michele_Bullock.pdf

🌐 Visiting: https://www.bis.org/review/r250728e.htm
🧾 Using filename base: 05_August_2025-Rajeshwar_Rao
📄 PDF found: https://www.bis.org/review/r250728e.pdf
📩 Downloaded PDF to ../output/2_2/downloads/05_August_2025-Rajeshwar_Rao.pdf

🌐 Visiting: https://www.bis.org/review/r250717g.htm
🧾 Using filename base: 05_August_2025-Chang_Yong_Rhee
📄 PDF found: https://www.bis.org/review/r250717g.pdf
📩 Downloaded PDF to ../output/2_2/downloa

# **3. Extracting text from PDF files**

After downloading the PDF files, we need to extract their textual content for analysis. This section covers:

1. Using the `PyPDF2` library to process PDF documents
2. Iterating through each downloaded PDF file in our directory
3. Extracting text content from all pages of each PDF
4. Saving the extracted text to corresponding text files in our designated text directory
5. Implementing error handling to manage potential issues in PDF processing

Text extraction is an essential step in the data pipeline as it converts the unstructured PDF content into plain text that can be more easily analyzed, processed, and used for tasks like natural language processing, sentiment analysis, or topic modeling. The `PyPDF2` library provides a straightforward way to extract text from PDF documents without requiring external dependencies.

In [10]:
for pdf_file in os.listdir(DOWNLOAD_DIR):
    if pdf_file.lower().endswith('.pdf'):
        pdf_path = DOWNLOAD_DIR / pdf_file
        print(f"\n👁️‍🗨️ Extracting: {pdf_path}")
        try:
            with open(pdf_path, "rb") as f:
                reader = PyPDF2.PdfReader(f)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""
            txt_filename = Path(pdf_file).stem + ".txt"
            txt_path = TEXT_DIR / txt_filename
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(text)
            print(f"📩 Saved text to {txt_path}")
        except Exception as e:
            print(f"❌ Error processing {pdf_path}: {e}")



👁️‍🗨️ Extracting: ../output/2_2/downloads/05_August_2025-Adnan_Zaylani_Mohamad_Zahid.pdf
📩 Saved text to ../output/2_2/texts/05_August_2025-Adnan_Zaylani_Mohamad_Zahid.txt

👁️‍🗨️ Extracting: ../output/2_2/downloads/10_July_2025-Yannis_Stournaras.pdf
📩 Saved text to ../output/2_2/texts/10_July_2025-Yannis_Stournaras.txt

👁️‍🗨️ Extracting: ../output/2_2/downloads/05_August_2025-Darryl_Chan.pdf
📩 Saved text to ../output/2_2/texts/05_August_2025-Darryl_Chan.txt

👁️‍🗨️ Extracting: ../output/2_2/downloads/30_July_2025-Rama_Krishna_Sithanen.pdf
📩 Saved text to ../output/2_2/texts/30_July_2025-Rama_Krishna_Sithanen.txt

👁️‍🗨️ Extracting: ../output/2_2/downloads/10_July_2025-Eli_M_Remolona.pdf
📩 Saved text to ../output/2_2/texts/10_July_2025-Eli_M_Remolona.txt

👁️‍🗨️ Extracting: ../output/2_2/downloads/05_August_2025-Jose_Luis_Escrivá.pdf
📩 Saved text to ../output/2_2/texts/05_August_2025-Jose_Luis_Escrivá.txt

👁️‍🗨️ Extracting: ../output/2_2/downloads/10_July_2025-Joachim_Nagel_1.pdf
📩 Saved 